In [1]:
import os
import random
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
TICK_DATA = os.path.join(os.getcwd(), '2.csv')
ORDER_DATA = os.path.join(os.getcwd(), 'MMOrderSet.csv')

tick = pd.read_csv(TICK_DATA, encoding='utf-16-le')
order = pd.read_csv(ORDER_DATA)

tick['DateTime'] = pd.to_datetime(tick['DateTime'], format="%m/%d/%Y %H:%M:%S.%f")
order['DateTime'] = pd.to_datetime(order['Time'])
order.drop(['Time'], axis=1, inplace=True)
order = order[['DateTime', 'Side']]

In [3]:
# tick = tick.iloc[tick.index%70 == 0]

In [4]:
tick.head()

,DateTime,Bid,Ask
0,2018-01-07 22:02:03.832,1.35560,1.35616
1,2018-01-07 22:02:03.845,1.35560,1.35617
2,2018-01-07 22:02:03.851,1.35554,1.35617
3,2018-01-07 22:02:03.859,1.35554,1.35618
4,2018-01-07 22:02:04.114,1.35554,1.35619


In [5]:
# tick = tick.sort_values(by=['DateTime'])
# tick.reset_index(drop=True, inplace=True)
# tick.head()

In [6]:
tick.shape

(1537052, 3)

In [7]:
order.head()

,DateTime,Side
0,2018-01-07 22:04:12,B
1,2018-01-07 22:04:13,S
2,2018-01-07 22:04:14,B
3,2018-01-07 22:04:15,S
4,2018-01-07 22:04:16,B


In [9]:
raw = pd.concat([tick, order], axis=0, sort=False).sort_values(by=['DateTime'])
raw.reset_index(drop=True, inplace=True)

In [ ]:
raw['Side'].fillna('T', inplace=True)
raw['Bid'].fillna('0', inplace=True)
raw['Ask'].fillna('0', inplace=True)

In [175]:
# flag = 1
# PnL = 0
# NetPos = 0
# CashPos = 0

K = 10 # open risk
L = 1 # Max PnL Loss
p = 0.5 # execution probability if at the market
q = 0.25 # follow-up order probability if executed
X = dt.timedelta(milliseconds=10) # reaction time

In [35]:
# M_Time = data.loc[0, 'DateTime']
# M_Bid = data.loc[0, 'Bid']
# M_Ask = data.loc[0, 'Ask']

# Q_Time = M_Time + X
# Q_Bid = M_Bid + 0.00001
# Q_Ask = M_Ask - 0.00001

# Q_Time_Last = Q_Time
# Q_Bid_Last = Q_Bid
# Q_Ask_Last = Q_Ask

# O_Time = 0

In [176]:
raw.loc[:, 'PnL'] = 0
raw.loc[:, 'Net Position'] = 0
raw.loc[:, 'Quote Bid'] = 0
raw.loc[:, 'Quote Ask'] = 0

In [191]:
def MarketMaking(data):
    
    flag = 1
    PnL = 0
    NetPos = 0
    CashPos = 0
    
    M_Time = data.loc[0, 'DateTime']
    M_Bid = data.loc[0, 'Bid']
    M_Ask = data.loc[0, 'Ask']

    Q_Time = M_Time + X
    Q_Bid = M_Bid + 0.00001
    Q_Ask = M_Ask - 0.00001

    Q_Time_Last = Q_Time
    Q_Bid_Last = Q_Bid
    Q_Ask_Last = Q_Ask

    O_Time = 0

    for i in range(1,len(data)):

        if data.loc[i, 'Side'] == 'T':

            Q_Time_Last = Q_Time
            Q_Bid_Last = Q_Bid
            Q_Ask_Last = Q_Ask

            M_Time = data.loc[i, 'DateTime']
            M_Bid = data.loc[i, 'Bid']
            M_Ask = data.loc[i, 'Ask']

            Q_Time = M_Time + X
            if flag == 2:
                Q_Bid = M_Bid + 0.00001
                Q_Ask = M_Ask - 0.00001
            elif flag == 0:
                Q_Bid = M_Bid
                Q_Ask = M_Ask
            elif flag == -1:
                Q_Bid = M_Bid + 0.00001
                Q_Ask = M_Ask + 0.00001
            elif flag == 1:
                Q_Bid = M_Bid - 0.00001
                Q_Ask = M_Ask - 0.00001

            if NetPos <= 0:
                data.loc[i, 'PnL'] = NetPos*M_Ask + CashPos
            else:
                data.loc[i, 'PnL'] = NetPos*M_Bid + CashPos
            data.loc[i, 'Net Position'] = NetPos

        else:
            if data.loc[i, 'Side'] == 'S':
                O_Time = data.loc[i, 'DateTime']

                if Q_Time_Last <= O_Time < Q_Time:
                    if Q_Bid_Last > M_Bid:
                        if random.random() <= q: 
                            NetPos += 2
                            CashPos -= 2*Q_Bid_last
                        else: 
                            NetPos += 1
                            CashPos -= Q_Bid_Last
                        PnL = NetPos*M_Bid + CashPos
                    if Q_Bid_Last == M_Bid:
                        if random.random() <= p:
                            NetPos += 1
                            CashPos -= Q_Bid_Last
                            PnL = NetPos*M_Bid + CashPos
                        else:
                            NetPos += 0
                            CashPos -= 0
                            PnL = NetPos*M_Bid + CashPos
                    if Q_Bid_Last < M_Bid:
                        NetPos += 0
                        CashPos -= 0
                        PnL = NetPos*M_Bid + CashPos

                if O_Time >= Q_Time:
                    if Q_Bid > M_Bid:
                        if random.random() <= q:
                            NetPos += 2
                            CashPos -= 2*Q_Bid
                        else: 
                            NetPos += 1
                            CashPos -= Q_Bid
                        PnL = NetPos*M_Bid + CashPos
                    if Q_Bid == M_Bid:
                        if random.random() <= p:
                            NetPos += 1
                            CashPos -= Q_Bid
                            PnL = NetPos*M_Bid + CashPos
                        else:
                            NetPos += 0
                            CashPos -= 0
                            PnL = NetPos*M_Bid + CashPos
                    if Q_Bid < M_Bid:
                        NetPos += 0
                        CashPos -= 0
                        PnL = NetPos*M_Bid + CashPos

            if data.loc[i, 'Side'] == 'B':
                O_Time = data.loc[i, 'DateTime']

                if Q_Time_Last <= O_Time < Q_Time:
                    if Q_Ask_Last < M_Ask:
                        if random.random() <= q: 
                            NetPos -= 2
                            CashPos += 2*Q_Ask_Last
                        else: 
                            NetPos -= 1
                            CashPos += Q_Ask_Last
                        PnL = NetPos*M_Ask + CashPos
                    if Q_Ask_Last == M_Ask:
                        if random.random() <= p:
                            NetPos -= 1
                            CashPos += Q_Ask_Last
                            PnL = NetPos*M_Ask + CashPos
                        else:
                            NetPos -= 0
                            CashPos += 0
                            PnL = NetPos*M_Ask + CashPos
                    if Q_Ask_Last > M_Ask:
                        NetPos -= 0
                        CashPos += 0
                        PnL = NetPos*M_Ask + CashPos

                if O_Time >= Q_Time:
                    if Q_Ask < M_Ask:
                        if random.random() <= q: 
                            NetPos -= 2
                            CashPos += 2*Q_Ask
                        else: 
                            NetPos -= 1
                            CashPos += Q_Ask
                        PnL =  NetPos*M_Ask + CashPos
                    if Q_Ask == M_Ask:
                        if random.random() <= p:
                            NetPos -= 1
                            CashPos += Q_Ask
                            PnL = NetPos*M_Ask + CashPos
                        else:
                            NetPos -= 0
                            CashPos += 0
                            PnL = NetPos*M_Ask + CashPos
                    if Q_Ask > M_Ask:
                        NetPos -= 0
                        CashPos += 0
                        PnL = NetPos*M_Ask + CashPos

            data.loc[i, 'PnL'] = PnL
            data.loc[i, 'Net Position'] = NetPos

        if -K/2 < NetPos < K/2:
            flag = 2
        elif (-K < NetPos <= -K/2) or (K/2 <= NetPos < K):
            flag = 0
        elif NetPos <= -K:
            flag = -1
        elif NetPos >= K:
            flag = 1


In [192]:
raw.head()

,DateTime,Bid,Ask,Side,PnL,Net Position,Quote Bid,Quote Ask
0,2018-01-07 22:02:03.832,1.3556,1.35616,T,0,0,0,0
1,2018-01-07 22:02:03.845,1.3556,1.35617,T,0,0,0,0
2,2018-01-07 22:02:03.851,1.35554,1.35617,T,0,0,0,0
3,2018-01-07 22:02:03.859,1.35554,1.35618,T,0,0,0,0
4,2018-01-07 22:02:04.114,1.35554,1.35619,T,0,0,0,0


In [193]:
t_delta = dt.timedelta(seconds=10)

START = order.groupby([order['DateTime'].dt.date]).min()
END = order.groupby([order['DateTime'].dt.date]).max()

In [194]:
for i in range(4):
    start = START.iloc[i, 0]
    end = END.iloc[i, 0]
    
    start -= t_delta
    end += t_delta

In [195]:
start

Timestamp('2018-01-12 08:43:35')

In [196]:
end

Timestamp('2018-01-12 08:45:17')

In [197]:
data = raw[(start <= raw['DateTime']) & (raw['DateTime'] <= end)]
data.reset_index(drop=True, inplace=True)

In [198]:
data.head()

,DateTime,Bid,Ask,Side,PnL,Net Position,Quote Bid,Quote Ask
0,2018-01-12 08:43:35.068,1.35747,1.35751,T,0,0,0,0
1,2018-01-12 08:43:35.206,1.35747,1.35752,T,0,0,0,0
2,2018-01-12 08:43:35.333,1.35747,1.35751,T,0,0,0,0
3,2018-01-12 08:43:35.366,1.35747,1.35752,T,0,0,0,0
4,2018-01-12 08:43:35.370,1.35746,1.35752,T,0,0,0,0


In [199]:
data.shape

(1337, 8)

In [200]:
MarketMaking(data)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


KeyboardInterrupt: 

In [ ]:
data['PnL'].plot()